In [16]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import QSystemTrayIcon
import tkinter as tk
import PIL.Image
import numpy as np
import cv2
import io
import json
import ctypes
from io import BytesIO
import win32clipboard
import glob
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision import types
from google.cloud import vision
import mss
import pyperclip
import mss.tools
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"

In [17]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()
    
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.types.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},
    )
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return texts[0].description

In [27]:
class SystemTrayIcon(QtWidgets.QSystemTrayIcon):

    def __init__(self, icon, parent=None):
        QtWidgets.QSystemTrayIcon.__init__(self, icon, parent)
        menu = QtWidgets.QMenu(parent)
        exitAction = menu.addAction("Exit", self.exit)
        self.setContextMenu(menu)
        self.show()

    def exit(self):
        print("icon exit")
        self.hide()
        QtCore.QCoreApplication.exit()

# def main(image):
#     app = QtWidgets.QApplication(sys.argv)
#     w = QtWidgets.QWidget()
#     trayIcon = SystemTrayIcon(QtGui.QIcon(image), w)
#     sys.exit(app.exec_())


# if __name__ == '__main__':
#     on='icon.ico'
#     main(on)
# main(r'icon.jpg')

In [37]:
from PyQt5.QtGui import QIcon
class MyWidget(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.setGeometry(0, 0, screen_width, screen_height)
        self.setWindowTitle(' ')
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )
        image = r'icon.jpg'
        self.tray_icon = SystemTrayIcon(QtGui.QIcon(image), self)
#         print(type(self))
#         print('Capture the screen...')
        self.show()

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        print("main esp exit")
        self.close()

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()

    def mouseReleaseEvent(self, event):
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())
        
        with mss.mss() as sct:
            # The screen part to capture
            monitor = {"top": y1, "left":x1, "width": abs(x2 - x1) , "height": abs(y2 - y1)}
            
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)

            sct_img = sct.grab(monitor)
            img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")

        img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
        im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
        ocr = dtlh(thresh, ["hi"])
#         send_text_clipboard(ocr)
#         print(ocr)

        cv2.imshow('Captured Image', img)
        cv2.waitKey(0)
        print("image exit")
        cv2.destroyAllWindows()

In [38]:
app = QtWidgets.QApplication(sys.argv)
window = MyWidget()
window.show()
app.aboutToQuit.connect(app.deleteLater)
sys.exit(app.exec_())

Capture the screen...
image exit


SystemExit: 0